#Grupo RM


1.   Ruy Ovidio

2.   Matheus Barreto


#Projeto Final - Parte 2

##Construção de um RAG utilizando LangChain


Deverá ser desenvolvido um sistema RAG (Retrieval-Augmented Generation) utilizando a biblioteca LangChain para um ou mais documentos de sua escolha. Podendo ser arquivos PDF, de texto, páginas da web etc. O projeto será avaliado nos seguintes aspectos:
Escolha do Documento
Splitting do Documento
Criação de Vector Store
Retrieval
Geração de Respostas

## Objetivo do trabalho

O objetivo deste trabalho consiste na construção de uma RAG utilizando LangChain para auxiliar um possivel candidato no concurso do BNDES, respondendo perguntas relacionadas ao edital do concurso.

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos da Web - Loading

In [ ]:
!pip install langchain_community langchain_openai faiss-gpu

In [ ]:
!pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader

uploaded = files.upload()

Saving edital-BNDES (1).pdf to edital-BNDES (1).pdf


In [ ]:
file_path = (
    "/content/edital-BNDES (1).pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load_and_split()

docs[0]

Document(metadata={'source': '/content/edital-BNDES (1).pdf', 'page': 0}, page_content='Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,\nque institui a Infraestrutura de Chaves Públicas Brasileira - ICP-Brasil.Este documento pode ser verificado no endereço eletrônico\nhttp://www.in.gov.br/autenticidade.html , pelo código 0530202407220002626\nNº 139, segunda-feira, 22 de julho de 2024 ISSN 1677-7069 Seção 3\nAVISO DE LICITAÇÃO\nPROCEDIMENTO LICITATÓRIO ELETRÔNICO Nº 91008/2024. Processo nº 091/2023.\nEsta Licitação é regida pela Lei nº 13.303, de 30/06/2016. Objeto: Concessão\nRemunerada de Uso para diversas Áreas Vagas do ETSP - Entreposto Terminal de São\nPaulo, conforme quantidades e especificações descritas no ANEXO I - TERMO DE\nREFERÊNCIA. Obtenção do Edital: a partir de 22/07/2024, através do site www.licitacoes-\ne.com.br, opção "Licitações" e na SELIC - Seção de Licitações. Visita: até 13/09/2024.\nSessão: em 16/09/2024 às 09h30 no site www.licitacoes-e.co

In [ ]:
print(docs[0].page_content[500:1000])

Concessão
Remunerada de Uso para diversas Áreas Vagas do ETSP - Entreposto Terminal de São
Paulo, conforme quantidades e especificações descritas no ANEXO I - TERMO DE
REFERÊNCIA. Obtenção do Edital: a partir de 22/07/2024, através do site www.licitacoes-
e.com.br, opção "Licitações" e na SELIC - Seção de Licitações. Visita: até 13/09/2024.
Sessão: em 16/09/2024 às 09h30 no site www.licitacoes-e.com.br.
MARIA VALDIRENE RODRIGUES DA SILVA CARLOS
Presidente da Comissão Julgadora
Ministério do Dese


### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

318


In [ ]:
print(all_splits[2].page_content)

do Documentário "Histórias da Fome no Brasil 2", com previsão de realização 
de abril de
2024 a maio de 2025.
Fundamento Legal: LEI 14.133/2021 - Artigo: 74 - Inciso: I. Vigência: 18/07/2024 a
17/08/2025. Valor Total: R$ 500.000,00. Data de Assinatura: 18/07/2024.
(COMPRASNET 4.0 - 19/07/2024).
EXTRATO DE TERMO ADITIVO Nº 2/2024 - UASG 550005
Número do Contrato: 33/2022.
Nº Processo: 71000.074238/2021-52.
Pregão. Nº 15/2022. Contratante: MINISTÉRIO DO DESENVOLVIMENTO E ASSISTÊ
NCIA
SOCIAL, FAMÍLIA E COMBATE À FOME. Contratado: 04.768.702/0001-70 - ENGEMIL-
ENGENHARIA, EMPREENDIMENTOS,MANUTENÇÃO E INSTALAÇÕES LTDA. Objeto: ALT
ERAR
o disposto no item 22.14 do termo de referência do Contrato Administrativo nº 33/2022
para inclusão de referência expressa ao Índice Nacional da Construção Civi l - INCC-DI como
critério de reajuste, passando o item vigorar com a seguinte redação:
"22.14 quando a repactuação solicitada pela Contratada se referir aos custos sujeitos à


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Quais são os conteudos basicos de língua portuguesa?")

print(retrieved_docs)

[Document(metadata={'source': '/content/edital-BNDES (1).pdf', 'page': 7, 'start_index': 3129}, page_content='Conceitos básicos; Tipos especiais; Representação; Caminhos e circuitos; Árvores e\nflorestas; Busca em largura; Busca em profundidade; Algoritmos de caminho mínimo;\nColoração de grafos; Algoritmos de árvore geradora mínima; Outros algoritmos. II -\nDESENVOLVIMENTO DE SISTEMAS: 1. Fundamentos de Linguagem de Programação: \nTipos\nde dados (vinculação; verificação de tipos, tipificação forte e fraca); Estruturas de controle\n(comandos de decisão e repetição, exceções, operadores ternários); Modularização; Sub-\nrotinas, funções e funções anônimas; Passagem de parâmetros por referênci\na e valor;\nEscopo de variáveis; Linguagem de montagem; Ligação (linking) estática e dinâmica;\nConceitos de processamento paralelo e distribuído; Programação assíncro\nna. 2.\nProgramação orientada a objetos: Conceitos de orientação a objetos; Classes e objetos;'), Document(metadata={'source': '/

#Adição

1 - Criação de Embeddings

2 - Indexação com FAISS

3 - Buscando Documentos

4 - Exibição dos Resultados

In [ ]:
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(all_splits, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Realiza a busca
query = "Quais são os conteúdos básicos de língua portuguesa?"
retrieved_docs = retriever.get_relevant_documents(query)

# Exibe os resultados
for idx, doc in enumerate(retrieved_docs):
    print(f"Documento {idx+1}:\n{doc.page_content[:500]}\n")


Documento 1:
Conceitos básicos; Tipos especiais; Representação; Caminhos e circuitos; Árvores e
florestas; Busca em largura; Busca em profundidade; Algoritmos de caminho mínimo;
Coloração de grafos; Algoritmos de árvore geradora mínima; Outros algoritmos. II -
DESENVOLVIMENTO DE SISTEMAS: 1. Fundamentos de Linguagem de Programação: 
Tipos
de dados (vinculação; verificação de tipos, tipificação forte e fraca); Estruturas de controle
(comandos de decisão e repetição, exceções, operadores ternários); Modulariza

Documento 2:
gramaticais relevantes. 3. Vocabulário. 4. Mecanismos de coesão textual (referenciação e
sequenciação). 5. Semântica.
CONHECIMENTOS TRANSVERSAIS: 1. Políticas Públicas e Desenvolvimento: 1.1
Planejamento governamental, finanças e gestão pública; 1.2 Papel das instituições; 1.3
Noções sobre finanças públicas, ciclo orçamentário, controles interno e e
xterno; 1.4
Noções básicas e estágios do ciclo político-administrativo da política pú blica; 1.4.1
Importância do monito

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


#Adição

No lugar de print(example_messages.to_messages()), adicionei uma conexão onde pode conectar a resposta à sua lógica de recuperação de documentos, garantindo que o contexto seja extraído dos documentos recuperados.

In [ ]:
# Recupera os documentos com base na pergunta
retrieved_docs = retriever.invoke("Quais são os conteúdos básicos de língua portuguesa?")

# Extrai o conteúdo dos documentos
context = "\n".join([doc.page_content for doc in retrieved_docs])

# Gera a resposta com base no contexto e na pergunta
example_messages = prompt_template.invoke({
    "context": context,
    "question": "Quais são os conteúdos básicos de língua portuguesa?"
})

print(example_messages.to_messages())


[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: Quais são os conteúdos básicos de língua portuguesa?\n\nContexto: Conceitos básicos; Tipos especiais; Representação; Caminhos e circuitos; Árvores e\nflorestas; Busca em largura; Busca em profundidade; Algoritmos de caminho mínimo;\nColoração de grafos; Algoritmos de árvore geradora mínima; Outros algoritmos. II -\nDESENVOLVIMENTO DE SISTEMAS: 1. Fundamentos de Linguagem de Programação: \nTipos\nde dados (vinculação; verificação de tipos, tipificação forte e fraca); Estruturas de controle\n(comandos de decisão e repetição, exceções, operadores ternários); Modularização; Sub-\nrotinas, funções e funções anônimas; Passagem de parâmetros por referênci\na e valor;\nEscopo de variáve

### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
query = input('Digite uma pergunta sobre o edital concurso do BNDES: ')
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Digite uma pergunta sobre o edital concurso do BNDES: Qual a data da prova?
A prova será realizada no dia 13/10/2024.

#Resumo do que foi realizado:

Escolha do Documento: Foi utilizado um PDF do edital do concurso do BNDES.

Splitting do Documento: Utilizamos o RecursiveCharacterTextSplitter para dividir o conteúdo dos documentos em partes menores, facilitando o processamento e a busca.

Criação de Vector Store: Foram Indexados os documentos usando FAISS e embeddings do OpenAI para criar uma base de dados vetorial eficiente para recuperação de informações.

Retrieval: Foi configurado um retriever para buscar e recuperar documentos relevantes com base em uma consulta, utilizando o tipo de busca por similaridade.

Geração de Respostas: Implementamos um pipeline para gerar respostas utilizando um modelo de linguagem (GPT-4(mini)) e um template de prompt específico.

No geral o projeto cobre todos os aspectos necessários para um sistema RAG e demonstra uma aplicação prática e funcional dos conceitos.


Aprimoramentos possíveis:

Explorar o ajuste fino dos parâmetros do retriever, testar diferentes modelos de linguagem ou adicionar funcionalidades adicionais, como o pré-processamento avançado dos documentos.